In [115]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
df_train=pd.read_csv('train.csv')
df_test=pd.read_csv('test.csv')
gender_submission=pd.read_csv('gender_submission.csv')

df=pd.concat([df_train,df_test])
sns.set()


In [116]:
print(df['Age'].describe())
df['Age']=df['Age'].fillna(df['Age'].median())
print("="*32)

count    1046.000000
mean       29.881138
std        14.413493
min         0.170000
25%        21.000000
50%        28.000000
75%        39.000000
max        80.000000
Name: Age, dtype: float64


In [117]:

print(df['Fare'].describe())
print(df['Fare'].mode())
df['Fare']=df['Fare'].fillna(df['Fare'].median())
print("="*32)

count    1308.000000
mean       33.295479
std        51.758668
min         0.000000
25%         7.895800
50%        14.454200
75%        31.275000
max       512.329200
Name: Fare, dtype: float64
0    8.05
Name: Fare, dtype: float64


In [118]:
print(df.Embarked.value_counts())
df['Embarked']=df['Embarked'].fillna('S')
df['Embarked']=df['Embarked'].map({'C':0,'Q':1,'S':2})

S    914
C    270
Q    123
Name: Embarked, dtype: int64


In [119]:

df['Sex']=df['Sex'].map({'male':1,'female':0})
df['SibSp']=df['SibSp'].apply(lambda x:np.power(x,1/100))


In [120]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split
Age=StandardScaler()
age=Age.fit_transform(df[['Age']])
df['Age']=age
df_train=df[:891]
x=df_train.drop(['Survived','Ticket','Cabin','Name','PassengerId'],axis=1)
y=df_train['Survived']
x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=0.25,random_state=42)
x.info()


<class 'pandas.core.frame.DataFrame'>
Int64Index: 891 entries, 0 to 890
Data columns (total 7 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   Pclass    891 non-null    int64  
 1   Sex       891 non-null    int64  
 2   Age       891 non-null    float64
 3   SibSp     891 non-null    float64
 4   Parch     891 non-null    int64  
 5   Fare      891 non-null    float64
 6   Embarked  891 non-null    int64  
dtypes: float64(3), int64(4)
memory usage: 55.7 KB


In [121]:
modelQ=GradientBoostingClassifier()
params={'loss':['log_loss', 'deviance', 'exponential'],'n_estimators':list(range(5,100,5))}     
model=GridSearchCV(modelQ,params,cv=5,scoring='accuracy').fit(x_train,y_train)

c:\Users\amavi\anaconda3\envs\PP_python\lib\site-packages\sklearn\model_selection\_validation.py:372: FitFailedWarning: 
95 fits failed out of a total of 285.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
95 fits failed with the following error:
Traceback (most recent call last):
  File "c:\Users\amavi\anaconda3\envs\PP_python\lib\site-packages\sklearn\model_selection\_validation.py", line 680, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "c:\Users\amavi\anaconda3\envs\PP_python\lib\site-packages\sklearn\ensemble\_gb.py", line 525, in fit
    self._check_params()
  File "c:\Users\amavi\anaconda3\envs\PP_python\lib\site-packages\sklearn\ensemble\_gb.py", line 282, in _check_params
    raise ValueError("Loss

In [122]:

df_test=df[891:1309]
x=df_test.drop(['Survived','Ticket','Cabin','Name','PassengerId'],axis=1)
predictions=model.predict(x)
gender_submission['Survived']=predictions


In [123]:
gender_submission.to_csv('gender_submission.csv')

In [124]:
pd.read_csv('gender_submission.csv',index_col=[0])


,PassengerId,Survived
0,892,0.0
1,893,0.0
2,894,0.0
3,895,0.0
4,896,1.0
...,...,...
413,1305,0.0
414,1306,1.0
415,1307,0.0
416,1308,0.0
